***
#### **For keep all lib**
`python -m pip freeze > requirements.txt`
#### **For install all lib**
`python -m pip install -r requirements.txt`
#### **For clean all lib use**
`python -m pip uninstall -y -r <(pip freeze)`

***
#### **install torch torchvision torchaudio**
`pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117`
#### **install detectron2**
`git clone https://github.com/facebookresearch/detectron2.git`
***
`python -m pip install -e detectron2`
***

In [ ]:
import torch, detectron2

In [ ]:
print(torch.cuda.is_available())

In [ ]:
!nvcc --version

In [ ]:
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__
# print(torch.cuda.is_available())
print("torch: ", TORCH_VERSION, "; cuda: ", torch.__version__)
print("detectron2:", detectron2.__version__)

In [ ]:
!nvidia-smi

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
# im = cv2.imread("./img/1.jpeg")
# im = cv2.imread("./img/2.jpeg")
im = cv2.imread("./img/cake/c_3.jpg")

# call imshow() using plt object
plt.imshow(im)

In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

In [ ]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
result_image = out.get_image()[:, :, ::-1]

plt.imshow(result_image)